In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

Using TensorFlow backend.


In [2]:
tickers = pd.read_csv('../data/frankfurt/FSE_metadata.csv')
tickers.head()

,code,name,description,refreshed_at,from_date,to_date
0,1COV_X,Covestro AG (1COV_X),Stock Prices for Covestro AG (1COV) from the F...,2019-02-15 22:30:19,2015-10-06,2019-02-15
1,2HR_X,H&R (2HR_X),Stock Prices for H&r Ag from the Frankfurt Sto...,2017-02-28 22:30:42,2000-01-03,2017-02-28
2,AAD_X,Amadeus FiRe AG (AAD_X),Stock Prices for Amadeus FiRe AG (AAD) from th...,2019-02-15 22:30:19,2000-02-01,2019-02-15
3,AB1_X,Air Berlin Plc (AB1_X),Stock Prices for Air Berlin Plc (AB1) from the...,2019-02-08 22:30:20,2006-05-11,2018-08-20
4,ADS_X,Adidas (ADS_X),Stock Prices for Adidas (ADS) from the Frankfu...,2019-02-15 22:30:19,2000-01-03,2019-02-15


In [3]:
choose_from = tickers["from_date"] < "2001-01-01"
choose_to = tickers["to_date"] > "2018-01-01"
tickers = tickers[choose_from & choose_to]
tickers.shape

(100, 6)

In [4]:
stock = pd.read_csv('../data/frankfurt/stocks/AAD_X.csv', index_col='Date', parse_dates=['Date'])
# stock = stock["Close"]
stock.head()

,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
Date,,,,,,,,,,
2000-02-01,NaN,NaN,NaN,14.0,NaN,0.0,NaN,NaN,NaN,0.0
2000-02-02,NaN,NaN,NaN,14.0,NaN,0.0,NaN,NaN,NaN,0.0
2000-02-03,NaN,NaN,NaN,14.6,NaN,0.0,NaN,NaN,NaN,0.0
2000-02-04,NaN,NaN,NaN,14.0,NaN,0.0,NaN,NaN,NaN,0.0
2000-02-07,NaN,NaN,NaN,14.0,NaN,0.0,NaN,NaN,NaN,0.0


In [5]:
stocks = pd.DataFrame(index=stock.index)
stocks = stocks.loc['2001-01-01':'2018-01-01']

In [6]:
stocks["AAD_X"] = stock["Close"]
stocks.head()

,AAD_X
Date,
2001-01-01,22.0
2001-01-02,22.3
2001-01-03,22.0
2001-01-04,22.0
2001-01-05,22.0


In [7]:
stock2 = pd.read_csv('../data/frankfurt/stocks/ADV_X.csv', index_col='Date', parse_dates=['Date'])
# stocks["ADS_X"] = stock2["Close"]
stock2.head()

,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
Date,,,,,,,,,,
2000-03-28,NaN,NaN,NaN,831.5,NaN,0.0,NaN,NaN,NaN,0.0
2000-03-29,NaN,NaN,NaN,810.0,NaN,0.0,NaN,NaN,NaN,0.0
2000-03-30,NaN,NaN,NaN,700.0,NaN,0.0,NaN,NaN,NaN,0.0
2000-03-31,NaN,NaN,NaN,648.0,NaN,0.0,NaN,NaN,NaN,0.0
2000-04-03,NaN,NaN,NaN,527.0,NaN,0.0,NaN,NaN,NaN,0.0


In [8]:
for ticker in tickers.code:
    stock = pd.read_csv(f'../data/frankfurt/stocks/{ticker}.csv', index_col='Date', parse_dates=['Date'])
    stocks[ticker] = stock["Close"].loc['2001-01-01':'2018-01-01']


In [9]:
stocks.head()

,AAD_X,ADS_X,ADV_X,AFX_X,AIR_X,AIXA_X,ALV_X,BAS_X,BAYN_X,BBZA_X,...,SZG_X,SZU_X,TEG_X,TKA_X,TTK_X,TUI1_X,UTDI_X,VIB3_X,VOS_X,VOW3_X
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-01,22.0,66.00,64.90,21.50,23.00,115.50,398.60,48.17,55.72,1.0,...,8.90,12.66,76.01,16.50,9.18,38.41,4.90,10.15,14.95,31.7
2001-01-02,22.3,65.00,52.15,22.41,22.95,104.00,387.42,48.70,56.90,1.0,...,8.90,12.22,70.00,17.14,9.50,39.23,4.38,11.10,14.95,31.7
2001-01-03,22.0,63.60,55.16,25.00,22.80,104.00,396.00,47.99,55.80,1.0,...,9.03,12.20,70.00,17.09,9.70,39.46,4.33,10.99,14.90,32.0
2001-01-04,22.0,66.00,48.60,26.00,22.59,110.95,388.00,49.06,53.50,1.0,...,9.35,12.13,70.00,17.50,9.40,39.91,4.05,11.85,15.52,32.6
2001-01-05,22.0,68.15,41.86,25.33,22.00,102.00,377.00,49.16,53.60,1.0,...,9.40,12.30,70.00,18.05,9.52,41.04,3.48,12.20,16.02,32.7


In [10]:
stocks.shape

(4213, 100)

In [13]:
stocks.isna().sum()

AAD_X        0
ADS_X       24
ADV_X       69
AFX_X        0
AIR_X        0
AIXA_X       0
ALV_X        0
BAS_X        0
BAYN_X       0
BBZA_X       1
BC8_X        0
BDT_X       12
BEI_X        0
BIO3_X       0
BMW_X        0
BOSS_X       0
BYW6_X     193
CBK_X        0
CEV_X        0
COK_X        0
COM_X       61
CON_X        0
COP_X      462
DAI_X        0
DBAN_X      76
DBK_X        1
DEZ_X       24
DLG_X        0
DPW_X        0
DRI_X        0
          ... 
PSM_X       24
PUM_X        0
QIA_X       24
QSC_X        0
RAA_X        0
RHK_X       62
RHM_X        0
RRTL_X    1433
RWE_X       62
SAP_X        0
SBS_X       13
SDF_X        0
SGL_X        0
SIE_X        0
SIX2_X       0
SKB_X        0
SLT_X      385
SOW_X        0
SPR_X      188
SRT3_X       2
SZG_X        0
SZU_X        0
TEG_X        4
TKA_X        0
TTK_X        0
TUI1_X      62
UTDI_X       0
VIB3_X      41
VOS_X        0
VOW3_X       0
Length: 100, dtype: int64

In [17]:
stocks.dropna(axis = 1).isna().sum()

AAD_X     0
AFX_X     0
AIR_X     0
AIXA_X    0
ALV_X     0
BAS_X     0
BAYN_X    0
BC8_X     0
BEI_X     0
BIO3_X    0
BMW_X     0
BOSS_X    0
CBK_X     0
CEV_X     0
COK_X     0
CON_X     0
DAI_X     0
DLG_X     0
DPW_X     0
DRI_X     0
DRW3_X    0
DTE_X     0
FME_X     0
FNTN_X    0
GBF_X     0
GIL_X     0
GWI1_X    0
HAW_X     0
HDD_X     0
HEI_X     0
         ..
IFX_X     0
INH_X     0
JUN3_X    0
LHA_X     0
LIN_X     0
MAN_X     0
MEO_X     0
MLP_X     0
MOR_X     0
MRK_X     0
NDA_X     0
PFV_X     0
PUM_X     0
QSC_X     0
RAA_X     0
RHM_X     0
SAP_X     0
SDF_X     0
SGL_X     0
SIE_X     0
SIX2_X    0
SKB_X     0
SOW_X     0
SZG_X     0
SZU_X     0
TKA_X     0
TTK_X     0
UTDI_X    0
VOS_X     0
VOW3_X    0
Length: 62, dtype: int64

In [18]:
stocks.isna().sum()

AAD_X        0
ADS_X       24
ADV_X       69
AFX_X        0
AIR_X        0
AIXA_X       0
ALV_X        0
BAS_X        0
BAYN_X       0
BBZA_X       1
BC8_X        0
BDT_X       12
BEI_X        0
BIO3_X       0
BMW_X        0
BOSS_X       0
BYW6_X     193
CBK_X        0
CEV_X        0
COK_X        0
COM_X       61
CON_X        0
COP_X      462
DAI_X        0
DBAN_X      76
DBK_X        1
DEZ_X       24
DLG_X        0
DPW_X        0
DRI_X        0
          ... 
PSM_X       24
PUM_X        0
QIA_X       24
QSC_X        0
RAA_X        0
RHK_X       62
RHM_X        0
RRTL_X    1433
RWE_X       62
SAP_X        0
SBS_X       13
SDF_X        0
SGL_X        0
SIE_X        0
SIX2_X       0
SKB_X        0
SLT_X      385
SOW_X        0
SPR_X      188
SRT3_X       2
SZG_X        0
SZU_X        0
TEG_X        4
TKA_X        0
TTK_X        0
TUI1_X      62
UTDI_X       0
VIB3_X      41
VOS_X        0
VOW3_X       0
Length: 100, dtype: int64